# Rent vs Buy
Should you rent or buy?

#### Load some useful packages

In [32]:
using Distributions, Formatting

#### Specify input values
These are user defined estimates. The current values are the default values used in the NYT rent vs buy calculator.

In [33]:
tourLength = 9 # how long you'll stay in years
oppRate = 4 # annual percent rate of return on money that can be invested or saved
infRate = 2 # annual inflation rate
taxProperty = 1.35 # property tax as percent of home price
taxMarginal = 20 # effective marginal tax rate including federal, state, and local
taxFees = 0 # percent of monthly common fees that are tax deductible
taxMarital = 1 # 0 if filing individual, 1 if filing joint
buy_price = 250_000 # home price in dollars
buy_mortgageRate = 3.67 # mortgage rate in percent
buy_down = 20 # down payment in percent of home price
buy_mortgageLength = 30 # length of mortgage in years
buy_growthRate = 3 # home price growth rate in percent per year
buy_closeBuyer = 4 # closing costs to buyer as percent home price
buy_closeSeller = 6 # closing costs to seller as percent home price
buy_mx = 1 # annual amount spent on maintenance, repairs, and renovations as a percent of purchase price
buy_insurance = 0.46 # annual homeowners insurance as a percent of home value
buy_utilities = 100 # monthly utilites that a landlord would cover
buy_fees = 0 # monthly condo, association, community fees
rent_growthRate = 2.5 # rental price growth rate in percent per year
rent_deposit = 1 # months of rent kept as security deposit
rent_broker = 0 # percent of annual rent charged as broker fee
rent_insurance = 1.32 # rental insurance as percent of annual rent;

#### Initialize variables
Set up the variables that we will use in our calculations.

In [34]:
# buy output vectors
buy_cost_initial = 0
buy_cost_recur = 0
buy_cost_opp = 0
buy_cost_proceeds = 0
buy_cost_total = 0 # total costs associated with buying after tourLength years

# rent output vectors
rent_cost_initial = 0
rent_cost_recur = 0
rent_cost_opp = 0
rent_cost_proceeds = 0
rent_cost_total = 0 # total costs associated with renting after tourLength years

# final output
eq_rent = 0 # if you can rent a similar home for less than this, renting is better

# intermediate variables
buy_mortgageMonthly = 0 # monthly mortgage 
debtMonthly = 0 # amount of mortgage debt paid in a given month
taxPropMonthly = 0 # monthly property taxes
buy_cost_monthly = 0 # monthly cost associated with buying;

#### Calculate NPV of Buying
First we calculate the initial costs

In [35]:
buy_cost_initial = ((buy_down + buy_closeBuyer) / 100) * buy_price

60000.0

Then we calculate the monthly mortgage payment

In [36]:
P = buy_price - (buy_down / 100) * buy_price # principal
r = (buy_mortgageRate / 100) / 12 # annual interest rate to months
n = buy_mortgageLength * 12 # number of monthly mortgage payments
loanBalance = P # total outstanding debt
buy_mortgageMonthly = P * (r * (1 + r) ^ n) / ((1 + r)^ n - 1)

917.17565301025

Then the recurring costs. 

(There might be a problem with the loan balance here...it doesn't match a loan calculator I found online)

In [37]:
for i = 1:(tourLength * 12)
  taxPropMonthly = ((taxProperty / 100) * buy_price / 12) * ((1 + (buy_growthRate / 100) / 12) ^ i)
  loanBalance = (P * (1 + r) ^ i) - (buy_mortgageMonthly / r) * ((1 + r) ^ i - 1)
  debtMonthly = loanBalance * (buy_mortgageRate / 100) / 12
  buy_cost_monthly = buy_mortgageMonthly + 
    buy_fees * ((1 + (infRate / 100) / 12) ^ i) + 
    buy_utilities * ((1 + (infRate / 100) / 12) ^ i) +
    ((buy_mx / 100) * buy_price / 12) * ((1 + ((buy_growthRate + infRate) / 100) / 12) ^ i) +
    ((buy_insurance / 100) * buy_price / 12) * ((1 + ((buy_growthRate + infRate) / 100) / 12) ^ i) +
    taxPropMonthly - 
    (taxPropMonthly + debtMonthly) * (taxMarginal / 100)
    buy_cost_recur = buy_cost_recur + (buy_cost_monthly / ((1 + ((oppRate - infRate) / 100) / 12) ^ i))
end

Then the opportunity cost.

(this cost also doesn't match the calculator results. changing the inflation rate affects this value a lot but not the value on the NYT calculator)

In [38]:
buy_cost_opp = (((buy_cost_initial + buy_cost_recur) * (1 + (oppRate - infRate) / 100) ^ tourLength -
                  (buy_cost_initial + buy_cost_recur)) * (1 - taxMarginal / 100)) * 
                  (1 / ((1 + (oppRate - infRate) / 100) ^ tourLength))

27891.30496117563

Then the proceeds from selling the house (which can be negative since it's defined as a cost)

In [39]:
buy_cost_proceeds = (-1) * 
    (buy_price * ((1 + buy_growthRate / 100) ^ tourLength ) * (1 - buy_closeSeller / 100) - loanBalance) * 
    (1 / ((1 + (oppRate - infRate) / 100) ^ tourLength))

-121872.31411297157

Now add up all the intermediate costs to get the total buying cost

In [40]:
buy_cost_total = round(buy_cost_initial + buy_cost_recur + buy_cost_opp + buy_cost_proceeds, 2)

119588.7

#### Results
Take the costs and the difference between our costs and the NYT calculator costs, then format them as dollars

In [41]:
# formatted costs
bci = format(buy_cost_initial, commas = true, precision = 2)
bcr = format(buy_cost_recur, commas = true, precision = 2)
bco = format(buy_cost_opp, commas = true, precision = 2)
bcp = format(buy_cost_proceeds, commas = true, precision = 2)
bct = format(buy_cost_total, commas = true, precision = 2)

#formatted cost differences
bci_d = format(buy_cost_initial - 60000, commas = true, precision = 2)
bcr_d = format(buy_cost_recur - 163398, commas = true, precision = 2)
bco_d = format(buy_cost_opp - 44587, commas = true, precision = 2)
bcp_d = format(buy_cost_proceeds + 145659, commas = true, precision = 2)
bct_d = format(buy_cost_total - 122337, commas = true, precision = 2);

Display the costs and differences.

(The difference here is pretty small but with other inputs the difference can be large)

In [43]:
println("Model Output: Buy")
println("Initial costs: \$$bci \t \t Diff: \$$bci_d")
println("Recurring costs: \$$bcr \t \t Diff: \$$bcr_d")
println("Opportunity costs: \$$bco \t \t Diff: \$$bco_d")
println("Net Proceeds: \$$bcp \t \t Diff: \$$bcp_d")
println("Total: \$$bct \t \t \t Diff: \$$bct_d")

Model Output: Buy
Initial costs: $60,000.00 	 	 Diff: $0.00
Recurring costs: $153,569.71 	 	 Diff: $-9,828.29
Opportunity costs: $27,891.30 	 	 Diff: $-16,695.70
Net Proceeds: $-121,872.31 	 	 Diff: $23,786.69
Total: $119,588.70 	 	 	 Diff: $-2,748.30
